In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [16]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Escolher as funções disponíveis

In [102]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

# Data model
class RouteQuery(BaseModel):
    """Encaminhe uma consulta de usuário para o datasource mais relevante."""

    datasource: Literal["vectorstore", "websearch", "agendamento"] = Field(
        ...,
        description="Dada uma pergunta do usuário, escolha encaminhá-la para websearch, vectorstore ou agendamento.",
    )

# LLM with function call
llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt 
system = """Você trabalha na Tech4.ai e é um especialista em encaminhar uma pergunta do usuário para um vectorstore, pesquisa na web ou agendamento de reunião. \n
O vectorstore contém documentos relacionados a perguntas comuns apenas sobre a empresa (Tech4.ai), tais como missão, visão, valores, cultura, programas internos, políticas \n
de trabalho remoto, horários, etc. Use o vectorstore para perguntas sobre esses tópicos. \n
Para perguntas sobre as ferramentas Github, Vscode, Jira e discord use a pesquisa na web. \n
Para perguntas sobre agendamento de reuniões use o agendamento."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

In [103]:
print(question_router.invoke({"question": "Qual a missão da Tech4.ai?"}))
print(question_router.invoke({"question": "Quero saber os valores da empresa"}))
print(question_router.invoke({"question": "Quero saber o que esperar de um dia de trabalho na Tech4.ai"}))
print(question_router.invoke({"question": "gostaria de agendar uma reunião com o time de desenvolvimento"}))
print(question_router.invoke({"question": "Como posso marcar uma reunião com o time de desenvolvimento?"}))
print(question_router.invoke({"question": "Preciso de ajuda com o Github"}))
print(question_router.invoke({"question": "Como posso usar o Vscode?"}))
print(question_router.invoke({"question": "Como posso usar o Jira?"}))

datasource='vectorstore'
datasource='vectorstore'
datasource='vectorstore'
datasource='agendamento'
datasource='agendamento'
datasource='websearch'
datasource='websearch'
datasource='websearch'


### Vector Database

In [104]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


# Load PDF document
pdf_path = "Base.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Split the combined documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs)

""" 
(VOLTAR SE SOBRAR TEMPO!)
O text_splitter não pega as imagens no final do pdf (Valores da Empresa). Tentativa de extrair texto das imagens:
# Extract text from images
image_paths = ["image1.PNG", "image2.PNG", "image3.PNG", "image4.PNG", "image5.PNG", "image6.PNG"]
image_texts = [extract_text_from_image(image_path) for image_path in image_paths]

doc_splitsIMAGES = text_splitter.split_documents(text_splitter.create_documents(image_texts)) 
"""

# Load an open-source embedding model from Hugging Face
embedding_function = SentenceTransformerEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Add to vector store
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="Empresa",
    embedding=embedding_function,
)

retriever = vectorstore.as_retriever()


###### Usar para deletar vector store

In [101]:

vectorstore = Chroma(collection_name="Empresa")
vectorstore.delete_collection()

###### .

In [105]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
 
# Data model
class GradeDocuments(BaseModel):
    """Pontuação binária para verificar a relevância nos documentos utilizados."""

    binary_score: str = Field(description="Documentos são relevantes para a pergunta, 'sim' or 'não'")

# LLM with function call 
structured_llm_grader_docs = llm.with_structured_output(GradeDocuments)

# Prompt 
system = """Você é um avaliador que avalia a relevância de um documento recuperado para uma pergunta do usuário. \n
 Se o documento contiver palavra(s)-chave ou significado semântico relacionado à pergunta, classifique-o como relevante. \n
 Dê uma pontuação binária "sim" ou "não" para indicar se o documento é relevante para a pergunta."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader_relevance = grade_prompt | structured_llm_grader_docs

In [106]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = ChatPromptTemplate.from_template(
    """Você é um assistente para tarefas de resposta a perguntas. Use as seguintes partes do contexto recuperado para responder à pergunta. \n
      Se você não souber a resposta, apenas diga que não sabe. Sempre responda em português do Brasil.
Question: {question}
Context: {context}
Answer:"""
)
 
# Chain
rag_chain = prompt | llm | StrOutputParser()

In [107]:
question = "Travel benefits é um programa interno?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader_relevance.invoke({"question": question, "document": doc_txt}))
# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

binary_score='sim'
Sim, Travel Benefits é um programa de benefícios de viagem que oferece descontos significativos e um orçamento para viagens, pago pela empresa.


### Hallucination Grader 

In [108]:
# Data model
class GradeHallucinations(BaseModel):
    """Pontuação binária para alucinação presente na resposta obtida."""

    binary_score: str = Field(description="Não considere a possibilidade de chamar APIs externas para obter informações adicionais. A resposta é apoiada pelos fatos, 'sim' ou 'não'.")
 
# LLM with function call 
structured_llm_grader_hallucination = llm.with_structured_output(GradeHallucinations)
 
# Prompt 
system = """Você é um avaliador que avalia se uma resposta gerada por LLM é apoiada por um conjunto de fatos recuperados. \n 
     Restrinja-se a dar uma pontuação binária, seja "sim" ou "não". Se a resposta for apoiada ou parcialmente apoiada pelo conjunto de fatos, considere-a um sim. \n
    Não considere a chamada de APIs externas para obter informações adicionais."""

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
]
)
  
hallucination_grader = hallucination_prompt | structured_llm_grader_hallucination
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='sim')

### Answer Grader 

In [ ]:
# Data model
class GradeAnswer(BaseModel):
    """Pontuação binária para avaliar se a resposta responde a pergunta."""

    binary_score: str = Field(description="Responde responde a pergunta, 'sim' ou 'não'")

# LLM with function call 
structured_llm_grader_answer = llm.with_structured_output(GradeAnswer)

# Prompt 
system = """Você é um avaliador que avalia se uma resposta aborda/resolve uma pergunta \n 
     Dê uma pontuação binária 'sim' ou 'não'. 'Sim' significa que a resposta resolve a pergunta."""

answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader_answer
answer_grader.invoke({"question": question,"generation": generation})